In [9]:
import numpy as np
import skimage

In [1]:
import numpy as np
from monai.data.image_reader import ImageReader, ITKReader
from ipywidgets.widgets import *
import ipywidgets as widgets

import matplotlib.pyplot as plt
import pytorch_lightning
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    ScaleIntensityRanged,
    Spacingd,
    EnsureType,
    EnsureChannelFirstd,
    RandFlipd,
    RandRotated,
    ToTensord,
    Resized,
    RandSpatialCropSamplesd,
    RandRotate90d,
    RandShiftIntensityd,
    KeepLargestConnectedComponent,
    RandCropByPosNegLabeld,
    RandCropByLabelClassesd
)

import wandb
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceFocalLoss, GeneralizedDiceLoss
from monai.losses import DiceLoss, DiceCELoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch, Dataset, LMDBDataset
from monai.config import print_config
from monai.apps import download_and_extract
import torch

import os
import glob
from tqdm import tqdm
import numpy as np
from monai.data import DataLoader
import os
import glob
from monai.transforms.spatial.array import Resize

from copy import deepcopy
from enum import Enum
from typing import Any, Dict, Hashable, List, Mapping, Optional, Sequence, Tuple, Union

from monai.config import DtypeLike, KeysCollection
from monai.config.type_definitions import NdarrayOrTensor
from monai.networks.layers import AffineTransform
from monai.networks.layers.simplelayers import GaussianFilter
from monai.transforms.croppad.array import CenterSpatialCrop, SpatialPad
from monai.transforms.inverse import InvertibleTransform
from monai.transforms.spatial.array import (
    Resize,
)
from monai.transforms.transform import MapTransform, RandomizableTransform
from monai.transforms.utils import create_grid
from monai.utils import (
    InterpolateMode,
    ensure_tuple_rep,
)
from monai.utils.deprecate_utils import deprecated_arg
from monai.utils.enums import TraceKeys
from monai.utils.module import optional_import
from monai.utils.type_conversion import convert_data_type, convert_to_dst_type
from monai.apps import load_from_mmar
from monai.apps.mmars import RemoteMMARKeys
from monai.networks.utils import copy_model_state
from monai.optimizers import generate_param_groups
import torch.optim as optim
import time
from pl_bolts.callbacks import PrintTableMetricsCallback
from monai.visualize import matshow3d, blend_images
import imageio
print_config()
from monai.losses import GeneralizedWassersteinDiceLoss
import random

MONAI version: 0.8.1
Numpy version: 1.22.3
Pytorch version: 1.11.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 71ff399a3ea07aef667b23653620a290364095b1

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.9.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.12.0
tqdm version: 4.64.0
lmdb version: 1.3.0
psutil version: 5.9.0
pandas version: 1.4.2
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [ ]:
trans = Compose([EnsureType(), AsDiscrete(to_onehot=3)])

In [ ]:
 post_pred = Compose([EnsureType(), AsDiscrete(argmax=True),KeepLargestConnectedComponent([1,2], is_onehot=False, independent=False)])

 [post_pred(i) for i in decollate_batch(prediction["prediction"]["output"])]
# liver_truth = [
#             1 if (post_label(i)[1,:,:,:].cpu() == 1).any() else 0 for i in decollate_batch(prediction['label'])
#         ]

test = prediction['output'].cpu()
prediction_1 = torch.argmax(test, dim=1)

liver_mask = (prediction_1 == 1).cpu()

prediction = np.ma.masked_array(test[:,2,:,:,:], mask=liver_mask)

unique_values = torch.unique(prediction_1)
predicted_class = 1 if 2 in unique_values else 0

In [11]:
img = np.array([[[0,1,1],[0,1,1],[0,1,0]],
                [[0,1,0],[0,1,1],[0,1,0]]])
labels= skimage.measure.label(img, return_num=False)
maxCC_nobcg = np.argmax(np.bincount(labels.flat, weights=img.flat))

maxCC_nobcg

1

In [ ]:
# Hay manchas azules en sitios que no son liver
# -> sol: largerst connected component del liver

# Hay manchas rojas en sitios que no son liver
# -> sol: remover cualquier mancha roja que no este en el liver


In [5]:
os.path.exists()

'/mnt/chansey/lauraalvarez/traumaAI/Liver_Segmentation'

In [6]:
import pickle

with open('/mnt/chansey/lauraalvarez/nnunet/nnUnet_preprocessed/Task504_LiverTrauma/dataset_properties.pkl', 'rb') as f:
    data = pickle.load(f)



In [7]:
data

{'all_sizes': [(1409, 512, 512),
  (1321, 512, 512),
  (1241, 512, 512),
  (1169, 512, 512),
  (1385, 512, 512),
  (1681, 512, 512),
  (2267, 512, 512),
  (626, 512, 512),
  (1313, 512, 512),
  (1337, 512, 512),
  (1458, 512, 512),
  (1217, 512, 512),
  (1345, 512, 512),
  (1261, 512, 512),
  (1377, 512, 512),
  (1328, 512, 512),
  (1089, 512, 512),
  (1433, 512, 512),
  (1401, 512, 512),
  (1569, 512, 512),
  (1369, 512, 512),
  (1513, 512, 512),
  (1337, 512, 512),
  (1385, 512, 512),
  (1361, 512, 512),
  (1513, 512, 512),
  (1449, 512, 512),
  (1321, 512, 512),
  (1281, 512, 512),
  (1369, 512, 512),
  (1433, 512, 512),
  (1337, 512, 512),
  (1406, 512, 512),
  (1369, 512, 512),
  (1265, 512, 512),
  (1066, 778, 378),
  (1657, 512, 512),
  (1289, 512, 512),
  (1401, 512, 512),
  (1417, 512, 512),
  (1201, 512, 512),
  (1361, 512, 512)],
 'all_spacings': [array([0.5       , 0.68199998, 0.68199998]),
  array([0.5       , 0.70200002, 0.70200002]),
  array([0.5       , 0.83899999, 0.83